# 2022年11月中文场景文字识别Baseline——基于PaddleOCR

## 常规赛简介
飞桨(PaddlePaddle)以百度多年的深度学习技术研究和业务应用为基础，集深度学习核心框架、基础模型库、端到端开发套件、工具组件和服务平台于一体，是中国首个开源开放、技术领先、功能完备的产业级深度学习平台。更多飞桨资讯，点击[此处](https://www.paddlepaddle.org.cn/)查看。

飞桨常规赛由百度飞桨于2019年发起，面向全球AI开发者，赛题范围广，涵盖领域多。常规赛旨在通过长期发布的经典比赛项目，为开发者提供学习锻炼机会，助力大家在飞将大赛中获得骄人成绩。

参赛选手需使用飞桨框架，基于特定赛题下的真实行业数据完成并提交任务。常规赛采取月度评比方式，为打破历史最高记录选手和当月有资格参与月度评奖的前10名选手提供飞桨特别礼包奖励。更多惊喜，更多收获，尽在飞桨常规赛。

## 数据集描述
本次赛题数据集共包括是2021年的8月份的数据集。提交的数据为1万条


为了避免标注上的歧义，竞赛组委会在计算指标之前对文字行进行如下预处理：
- 全角统一为半角
- 英文字符统一为小写
- 中文字符统一为简体
- 忽略所有空格和符号

# 快速开始
PaddleOCR旨在打造一套丰富、领先、且实用的OCR工具库，助力使用者训练出更好的模型，并应用落地。该基线使用PaddleOCR的推理模型，助力开发者快速完成结果文件的生成。

悄咪咪告诉你：用 **hub.fastgit.org** 代替 **github.com** 会有意想不到的效果哦~~
```
! git clone https://hub.fastgit.org/PaddlePaddle/PaddleOCR
! pip install -r PaddleOCR/requirements.txt
```
考虑到提交结果文件的需要，我将 `PaddleOCR/tools/infer/predict_rec.py` 进行了修改，包括返回结果的后处理（全角转半角、英文大写转小写、删除符合和空格）以及结果文件的上交格式，如果重新clone仓库需要再一次修改，比较麻烦。

In [2]:
! wget -P model/server/infer https://paddleocr.bj.bcebos.com/20-09-22/server/rec/ch_ppocr_server_v1.1_rec_infer.tar && \
  cd model/server/infer && \
  tar xf ch_ppocr_server_v1.1_rec_infer.tar && \
  rm ch_ppocr_server_v1.1_rec_infer.tar

--2021-09-10 09:42:35--  https://paddleocr.bj.bcebos.com/20-09-22/server/rec/ch_ppocr_server_v1.1_rec_infer.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 182.61.200.195, 182.61.200.229, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|182.61.200.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94362069 (90M) [application/x-tar]
Saving to: ‘model/server/infer/ch_ppocr_server_v1.1_rec_infer.tar’

ch_ppocr_server_v1. 100%[===================>]  89.99M  30.8MB/s    in 2.9s    

2021-09-10 09:42:38 (30.8 MB/s) - ‘model/server/infer/ch_ppocr_server_v1.1_rec_infer.tar’ saved [94362069/94362069]



## 解压数据
AI Studio的机制会在每次启动项目的时候把 data 目录下的文件初始化。为了优化启动速度，我们可以将内存占用大的数据解压到 data 目录下，内存占用小的文件放到 work 目录下，这样每次启动时只需花费解压一次数据的时间，更快完成项目加载。

In [3]:
! cd data && \
  unzip -qo data62842/train_images.zip && \
  unzip -qo data62843/test_images.zip && \
  cp data62842/train_label.csv . && \
  rm -r __MACOSX

## 模型预测

In [4]:
! cd PaddleOCR && \
  python tools/infer/predict_rec.py \
  --image_dir="../data/test_images" \
  --rec_model_dir="../model/server/infer/ch_ppocr_server_v1.1_rec_infer"

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
W0910 09:42:56.583988   467 analysis_predictor.cc:1145] Deprecated. Please use CreatePredictor instead.
Total predict time for 10000 images:48.519


# 继续冲刺
赛程赛制中有写道：**每月1-5日公布上一个月总榜。高于评奖分数（85分）且排名前10名选手可获得飞桨周边特别礼包（飞桨周边+100小时算力卡）。** 我们使用中英文通用OCR模型，可以得到82左右的分数，这显然不满足评奖条件，继续改进模型是有必要的。

在上面的模型预测中，我们使用的是PaddleOCR提供的推理模型，520W左右的数据集（真实数据26W+合成数据500W）训练，能够较为笼统地概括中文场景的文字特点。然而，为了针对常规赛的要求，我们可以在原模型的基础上用训练集微调，进一步提升我们模型的准确率~

在PaddleOCR预训练模型的基础上做微调，大体需要下面几步：
1. 准备训练数据
	- 图片
   - 标签
   - 词典
2. 修改配置文件
	- 训练配置
   - 数据读取
3. 下载模型
***
**注意：**
- 训练过程中保存的checkpoints模型（预训练模型）结构为 `*.pdmodel` `*.pdopt` `*.pdparams`
- inference模型结构为 `model` `params`

更多介绍：[PaddleOCR整体目录结构](https://github.com/PaddlePaddle/PaddleOCR/blob/develop/doc/doc_ch/tree.md#%E6%95%B4%E4%BD%93%E7%9B%AE%E5%BD%95%E7%BB%93%E6%9E%84)

## 数据准备
图像数据已经解压过，就不重复了，这里我们主要谈谈标签的预处理。上面说过，为了避免标注上的歧义，竞赛组委会在计算指标之前对文字行进行过如下预处理：
- 全角统一为半角
- 英文字符统一为小写
- 中文字符统一为简体
- 忽略所有空格和符号
***
`train_label.csv` 文件格式如下：
| name | value |
| :-: | :-: | 
| 0.jpg     | 拉拉     | 
| 1.jpg | ６号 |
| 2.jpg | 胖胖 |
| ... | ... |
| 49999.jpg | 脑维修 |


我们使用Pandas读取 train_label.csv 文件，对每一行的标注信息进行处理，并且删除处理后为空字符的标注，将其保存为 .txt 文件。

In [5]:
import pandas as pd

def Q2B(s):
    """全角转半角"""
    inside_code=ord(s)
    if inside_code==0x3000:
        inside_code=0x0020
    else:
        inside_code-=0xfee0
    if inside_code<0x0020 or inside_code>0x7e: #转完之后不是半角字符返回原来的字符
        return s
    return chr(inside_code)

def stringQ2B(s):
    """把字符串全角转半角"""
    return "".join([Q2B(c) for c in s])

def is_chinese(s):
    """判断unicode是否是汉字"""
    for c in s:
        if c < u'\u4e00' or c > u'\u9fa5':
            return False
    return True

def is_number(s):
    """判断unicode是否是数字"""
    for c in s:
        if c < u'\u0030' or c > u'\u0039':
            return False
    return True

def is_alphabet(s):
    """判断unicode是否是英文字母"""
    for c in s:
        if c < u'\u0061' or c > u'\u007a':
            return False
    return True

def del_other(s):
    """判断是否非汉字，数字和小写英文"""
    res = str()
    for c in s:
        if not (is_chinese(c) or is_number(c) or is_alphabet(c)):
            c = ""
        res += c
    return res


df = pd.read_csv("data/train_label.csv", encoding="gbk")
name, value = list(df.name), list(df.value)
for i, label in enumerate(value):
    # 全角转半角
    label = stringQ2B(label)
    # 大写转小写
    label = "".join([c.lower() for c in label])
    # 删除所有空格符号
    label = del_other(label)
    value[i] = label

# 删除标签为""的行
data = zip(name, value)
data = list(filter(lambda c: c[1]!="", list(data)))
# 保存到work目录
with open("data/train_label.txt", "w") as f:
    for line in data:
        f.write(line[0] + "\t" + line[1] + "\n")

字典在 `ppocr/utils/ppocr_keys_v1.txt` 中有提供，是一个包含6623个字符的中文字典。字典有什么用呢？通过字典，可以在训练的时候把字符映射成字符的索引。这里针对训练集，写了一个3096个字符的字典，开发者可以按需选择。

In [6]:
def create_label_list(train_list):
    classSet = set()
    with open(train_list) as f:
        next(f)
        for line in f:
            img_name, label = line.strip().split("\t")
            for e in label:
                classSet.add(e)
    # 在类的基础上加一个blank
    classList = sorted(list(classSet))
    with open("data/label_list.txt", "w") as f:
        for c in classList:
            f.write("{}\n".format(c))

create_label_list("data/train_label.txt")

## 下载模型
在 [文本识别算法](https://github.com/PaddlePaddle/PaddleOCR/blob/develop/doc/doc_ch/algorithm_overview.md#2%E6%96%87%E6%9C%AC%E8%AF%86%E5%88%AB%E7%AE%97%E6%B3%95)
 中能找到很多预训练模型，这些模型使用MJSynth和SynthText两个文字识别数据集训练。针对本次常规赛，可以使用训练集对模型进行微调。以CRNN模型为例，下面的代码可以将预训练模型下载到AI Studio：

In [7]:
! wget -P model https://paddleocr.bj.bcebos.com/rec_r34_vd_none_bilstm_ctc.tar && \
  cd model && \
  tar xf rec_r34_vd_none_bilstm_ctc.tar && \
  rm rec_r34_vd_none_bilstm_ctc.tar

--2021-09-10 09:44:55--  https://paddleocr.bj.bcebos.com/rec_r34_vd_none_bilstm_ctc.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 182.61.200.195, 182.61.200.229, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|182.61.200.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 421031424 (402M) [application/x-tar]
Saving to: ‘model/rec_r34_vd_none_bilstm_ctc.tar’

rec_r34_vd_none_bil 100%[===================>] 401.53M  42.7MB/s    in 7.7s    

2021-09-10 09:45:03 (52.1 MB/s) - ‘model/rec_r34_vd_none_bilstm_ctc.tar’ saved [421031424/421031424]



## 输出训练结果
下载完预训练模型后，可通过 -c 参数选择 PaddleOCR/configs/rec/ 路径下的多种模型配置进行训练。为了让大家熟悉PaddleOCR的使用，训练配置就留给大家自己选择，不在再多做介绍。在完成训练配置后，就可以开始训练了！

[PaddleOCR主页](https://github.com/PaddlePaddle/PaddleOCR) 如果repo对你能够帮助，可以点一个star支持一下呀~~

大家在提交结果文件的时候需要注意，我们的 `*.txt` 文件需要以下格式（**注意第一行**）：

| new_name | value | 
| :-: | :-: | 
| 0.jpg | 识别结果0 | 
| 1.jpg | 识别结果1 |
| 2.jpg | 识别结果2 |
|  ...  |  ...    |
| 9999.jpg | 识别结果9999 |




In [8]:
! cd PaddleOCR && \
  python tools/infer/predict_rec.py \
  --image_dir="../data/test_images" \
  --rec_model_dir="../model/server/infer/ch_ppocr_server_v1.1_rec_infer"

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
W0910 09:45:11.164461   636 analysis_predictor.cc:1145] Deprecated. Please use CreatePredictor instead.
Total predict time for 10000 images:48.598
